# Pseudo-labeling for Distribution Adaptation

**Strategy**: Use confident test predictions to augment training data, adapting the model to the test distribution.

**Hypothesis**: The CV-LB gap is due to distribution shift (test solvents are different from training). Pseudo-labeling can help the model adapt to the test distribution.

**Implementation**:
1. Train initial model on training data
2. Make predictions on test data (held-out fold)
3. Select high-confidence predictions (low variance across ensemble)
4. Add pseudo-labeled samples to training
5. Retrain model on augmented data
6. Repeat for a few iterations

**Model**: CatBoost+XGBoost ensemble (best performing tabular model)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# CatBoost + XGBoost Ensemble with Pseudo-labeling
import catboost as cb
import xgboost as xgb
import tqdm

class PseudoLabelingCatXGBEnsemble:
    def __init__(self, data='single', n_iterations=3, confidence_threshold=0.8):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = FullFeaturizer(mixed=self.mixed)
        self.scalers = [StandardScaler() for _ in range(3)]
        self.catboost_models = []
        self.xgb_models = []
        self.cat_weight = 0.5
        self.xgb_weight = 0.5
        self.n_iterations = n_iterations
        self.confidence_threshold = confidence_threshold
        
    def _train_base_models(self, X_feats, Y_vals):
        """Train base CatBoost and XGBoost models"""
        self.catboost_models = []
        self.xgb_models = []
        
        for i in range(3):
            y_scaled = self.scalers[i].fit_transform(Y_vals[:, i:i+1]).ravel()
            
            # CatBoost
            cat_model = cb.CatBoostRegressor(
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3,
                random_seed=42,
                verbose=False
            )
            cat_model.fit(X_feats, y_scaled)
            self.catboost_models.append(cat_model)
            
            # XGBoost
            xgb_model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_lambda=3,
                random_state=42,
                verbosity=0
            )
            xgb_model.fit(X_feats, y_scaled)
            self.xgb_models.append(xgb_model)
    
    def _predict_with_uncertainty(self, X_feats):
        """Predict with uncertainty estimation (variance between CatBoost and XGBoost)"""
        preds = []
        uncertainties = []
        
        for i in range(3):
            cat_pred = self.catboost_models[i].predict(X_feats)
            xgb_pred = self.xgb_models[i].predict(X_feats)
            
            # Ensemble prediction
            pred_scaled = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
            pred = self.scalers[i].inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
            preds.append(pred)
            
            # Uncertainty = absolute difference between models
            uncertainty = np.abs(cat_pred - xgb_pred)
            uncertainties.append(uncertainty)
        
        return np.stack(preds, axis=1), np.stack(uncertainties, axis=1)
        
    def train_model(self, X, Y):
        """Train with pseudo-labeling iterations"""
        X_feats = self.featurizer.featurize(X)
        Y_vals = Y.values
        
        # Initial training
        self._train_base_models(X_feats, Y_vals)
        
        # Note: In the actual CV loop, we don't have access to test data
        # So pseudo-labeling is done within the training data using a held-out set
        # This is a simplified version that just trains normally
        # The real benefit would come from using unlabeled test data
    
    def predict(self, X):
        X_feats = self.featurizer.featurize(X)
        preds, _ = self._predict_with_uncertainty(X_feats)
        return torch.tensor(preds)

print('PseudoLabelingCatXGBEnsemble defined')

PseudoLabelingCatXGBEnsemble defined


In [6]:
# Actually, pseudo-labeling in CV is tricky because we don't have access to test data
# Let's try a different approach: Self-training within the training data
# We'll use a held-out portion of training data as "pseudo-test" data

class SelfTrainingCatXGBEnsemble:
    def __init__(self, data='single', n_iterations=2, holdout_ratio=0.2):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = FullFeaturizer(mixed=self.mixed)
        self.scalers = [StandardScaler() for _ in range(3)]
        self.catboost_models = []
        self.xgb_models = []
        self.cat_weight = 0.5
        self.xgb_weight = 0.5
        self.n_iterations = n_iterations
        self.holdout_ratio = holdout_ratio
        
    def _train_base_models(self, X_feats, Y_vals):
        """Train base CatBoost and XGBoost models"""
        self.catboost_models = []
        self.xgb_models = []
        self.scalers = [StandardScaler() for _ in range(3)]
        
        for i in range(3):
            y_scaled = self.scalers[i].fit_transform(Y_vals[:, i:i+1]).ravel()
            
            # CatBoost
            cat_model = cb.CatBoostRegressor(
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3,
                random_seed=42,
                verbose=False
            )
            cat_model.fit(X_feats, y_scaled)
            self.catboost_models.append(cat_model)
            
            # XGBoost
            xgb_model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_lambda=3,
                random_state=42,
                verbosity=0
            )
            xgb_model.fit(X_feats, y_scaled)
            self.xgb_models.append(xgb_model)
    
    def _predict_raw(self, X_feats):
        """Predict without uncertainty"""
        preds = []
        for i in range(3):
            cat_pred = self.catboost_models[i].predict(X_feats)
            xgb_pred = self.xgb_models[i].predict(X_feats)
            pred_scaled = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
            pred = self.scalers[i].inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
            preds.append(pred)
        return np.stack(preds, axis=1)
        
    def train_model(self, X, Y):
        """Train with self-training iterations"""
        X_feats = self.featurizer.featurize(X)
        Y_vals = Y.values
        
        # Split into train and holdout
        n = len(X_feats)
        n_holdout = int(n * self.holdout_ratio)
        indices = np.random.permutation(n)
        train_idx = indices[n_holdout:]
        holdout_idx = indices[:n_holdout]
        
        X_train = X_feats[train_idx]
        Y_train = Y_vals[train_idx]
        X_holdout = X_feats[holdout_idx]
        Y_holdout = Y_vals[holdout_idx]
        
        # Initial training on subset
        self._train_base_models(X_train, Y_train)
        
        # Self-training iterations
        for iteration in range(self.n_iterations):
            # Predict on holdout
            holdout_preds = self._predict_raw(X_holdout)
            
            # Combine original training data with pseudo-labeled holdout
            X_combined = np.vstack([X_train, X_holdout])
            Y_combined = np.vstack([Y_train, holdout_preds])
            
            # Retrain on combined data
            self._train_base_models(X_combined, Y_combined)
        
        # Final training on all data
        self._train_base_models(X_feats, Y_vals)
    
    def predict(self, X):
        X_feats = self.featurizer.featurize(X)
        preds = self._predict_raw(X_feats)
        return torch.tensor(preds)

print('SelfTrainingCatXGBEnsemble defined')

SelfTrainingCatXGBEnsemble defined


In [ ]:
# Cross-validation on single solvent data
X, Y = load_data("single_solvent")
print(f'Single solvent data: {len(X)} samples, {len(X["SOLVENT NAME"].unique())} solvents')

all_mse = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_out_splits(X, Y))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SelfTrainingCatXGBEnsemble(data='single', n_iterations=2, holdout_ratio=0.2)
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse.append(mse)

single_mse = np.mean(all_mse)
print(f'\nSingle Solvent MSE: {single_mse:.6f}')

In [ ]:
# Cross-validation on full data
X_full, Y_full = load_data("full")
print(f'Full data: {len(X_full)} samples')

all_mse_full = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SelfTrainingCatXGBEnsemble(data='full', n_iterations=2, holdout_ratio=0.2)
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse_full.append(mse)

full_mse = np.mean(all_mse_full)
print(f'\nFull Data MSE: {full_mse:.6f}')

In [ ]:
# Calculate overall CV score
n_single = 656
n_full = 1227  # Corrected from previous wrong value
overall_mse = (single_mse * n_single + full_mse * n_full) / (n_single + n_full)

print(f'\n=== Self-Training Results ===')
print(f'Single Solvent MSE: {single_mse:.6f}')
print(f'Full Data MSE: {full_mse:.6f}')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest baseline (CatBoost+XGBoost): 0.008092')
print(f'Difference: {(overall_mse - 0.008092) / 0.008092 * 100:.2f}%')

In [ ]:
# Let's also try a simpler approach: Conservative predictions
# Blend predictions toward the training mean when uncertainty is high

class ConservativeCatXGBEnsemble:
    def __init__(self, data='single', blend_factor=0.1):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = FullFeaturizer(mixed=self.mixed)
        self.scalers = [StandardScaler() for _ in range(3)]
        self.catboost_models = []
        self.xgb_models = []
        self.cat_weight = 0.5
        self.xgb_weight = 0.5
        self.blend_factor = blend_factor
        self.train_means = None
        
    def train_model(self, X, Y):
        X_feats = self.featurizer.featurize(X)
        Y_vals = Y.values
        
        # Store training means for conservative blending
        self.train_means = Y_vals.mean(axis=0)
        
        self.catboost_models = []
        self.xgb_models = []
        
        for i in range(3):
            y_scaled = self.scalers[i].fit_transform(Y_vals[:, i:i+1]).ravel()
            
            # CatBoost
            cat_model = cb.CatBoostRegressor(
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3,
                random_seed=42,
                verbose=False
            )
            cat_model.fit(X_feats, y_scaled)
            self.catboost_models.append(cat_model)
            
            # XGBoost
            xgb_model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_lambda=3,
                random_state=42,
                verbosity=0
            )
            xgb_model.fit(X_feats, y_scaled)
            self.xgb_models.append(xgb_model)
    
    def predict(self, X):
        X_feats = self.featurizer.featurize(X)
        preds = []
        
        for i in range(3):
            cat_pred = self.catboost_models[i].predict(X_feats)
            xgb_pred = self.xgb_models[i].predict(X_feats)
            
            # Ensemble
            pred_scaled = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
            pred = self.scalers[i].inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
            
            # Conservative blending toward training mean
            pred = (1 - self.blend_factor) * pred + self.blend_factor * self.train_means[i]
            preds.append(pred)
        
        return torch.tensor(np.stack(preds, axis=1))

print('ConservativeCatXGBEnsemble defined')

In [ ]:
# Test conservative predictions with different blend factors
X, Y = load_data("single_solvent")

for blend_factor in [0.0, 0.05, 0.1, 0.15, 0.2]:
    all_mse = []
    for fold_idx, split in enumerate(generate_leave_one_out_splits(X, Y)):
        (train_X, train_Y), (test_X, test_Y) = split
        
        model = ConservativeCatXGBEnsemble(data='single', blend_factor=blend_factor)
        model.train_model(train_X, train_Y)
        
        preds = model.predict(test_X).numpy()
        targets = test_Y.values
        
        mse = np.mean((preds - targets) ** 2)
        all_mse.append(mse)
    
    print(f'Blend factor {blend_factor:.2f}: Single Solvent MSE = {np.mean(all_mse):.6f}')

In [ ]:
# The baseline (blend_factor=0) should match our best model
# Let's verify and then run the full CV with the best blend factor

# First, let's run the standard CatBoost+XGBoost ensemble to verify baseline
class CatXGBEnsemble:
    def __init__(self, data='single'):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = FullFeaturizer(mixed=self.mixed)
        self.scalers = [StandardScaler() for _ in range(3)]
        self.catboost_models = []
        self.xgb_models = []
        self.cat_weight = 0.5
        self.xgb_weight = 0.5
        
    def train_model(self, X, Y):
        X_feats = self.featurizer.featurize(X)
        Y_vals = Y.values
        
        self.catboost_models = []
        self.xgb_models = []
        
        for i in range(3):
            y_scaled = self.scalers[i].fit_transform(Y_vals[:, i:i+1]).ravel()
            
            # CatBoost
            cat_model = cb.CatBoostRegressor(
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3,
                random_seed=42,
                verbose=False
            )
            cat_model.fit(X_feats, y_scaled)
            self.catboost_models.append(cat_model)
            
            # XGBoost
            xgb_model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_lambda=3,
                random_state=42,
                verbosity=0
            )
            xgb_model.fit(X_feats, y_scaled)
            self.xgb_models.append(xgb_model)
    
    def predict(self, X):
        X_feats = self.featurizer.featurize(X)
        preds = []
        
        for i in range(3):
            cat_pred = self.catboost_models[i].predict(X_feats)
            xgb_pred = self.xgb_models[i].predict(X_feats)
            
            # Ensemble
            pred_scaled = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
            pred = self.scalers[i].inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
            preds.append(pred)
        
        return torch.tensor(np.stack(preds, axis=1))

print('CatXGBEnsemble defined')

In [ ]:
# Run full CV with baseline CatXGBEnsemble
X, Y = load_data("single_solvent")
print(f'Single solvent data: {len(X)} samples')

all_mse = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_out_splits(X, Y))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = CatXGBEnsemble(data='single')
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse.append(mse)

single_mse_baseline = np.mean(all_mse)
print(f'\nBaseline Single Solvent MSE: {single_mse_baseline:.6f}')

In [ ]:
# Run full CV on full data
X_full, Y_full = load_data("full")
print(f'Full data: {len(X_full)} samples')

all_mse_full = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = CatXGBEnsemble(data='full')
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse_full.append(mse)

full_mse_baseline = np.mean(all_mse_full)
print(f'\nBaseline Full Data MSE: {full_mse_baseline:.6f}')

In [ ]:
# Calculate overall baseline CV score
n_single = 656
n_full = 1227
overall_mse_baseline = (single_mse_baseline * n_single + full_mse_baseline * n_full) / (n_single + n_full)

print(f'\n=== Baseline CatXGBEnsemble Results ===')
print(f'Single Solvent MSE: {single_mse_baseline:.6f}')
print(f'Full Data MSE: {full_mse_baseline:.6f}')
print(f'Overall MSE: {overall_mse_baseline:.6f}')
print(f'\nExpected (from exp_049): 0.008092')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = CatXGBEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = CatXGBEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################